## Emily's state transition function

In [18]:
import Pkg
Pkg.add("Plots")
using Plots

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


In [19]:
import Pkg
Pkg.add("Distributions")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


In [20]:
using CSV, Random, DataFrames
using GLMakie
using GeoDatasets

In [21]:
include("../dev-yuji/heuristics.jl")
include("state.jl")

TR_orbit_clean (generic function with 2 methods)

In [6]:
dataset = "../src/obs_site_Earth_50.csv"

"../src/obs_site_Earth_50.csv"

In [23]:
# constants
Re = 6371;
mu_E = 3.986004418e5; #km^3/m^2  
slew_limit = 15

15

## MCTS Functions

In [11]:
# A shortened representation of MDP that works for online planning (no full state-space, TR captures T and R)
struct MDP 
    gamma # Discount factor
    A # Action space
    TR # Transition model
end 

In [12]:
function rollout(P, s, d)
    s_ = copy(s)
    ret = 0.0 
    for t in 1:d
        a = rand(P.A)
        # a = heuristic_action(s, slew_limit)
        s_, r = P.TR(s_, a)
        ret += P.gamma^(t-1) * r
    end 
    return ret 
end

struct MonteCarloTreeSearch
    P
    N   # visit count
    Q   # action-value estimate
    d   # depth to go through for the Monte Carlo search
    d_r # depth to go through the rollout at the end of the MCTS
    m   # num of simulations
    c   # exploration constant 
    U   # value function estimate 
end 

function (π::MonteCarloTreeSearch)(s)
    for k in 1:π.m
        simulate!(π,s)
    end 
    return argmax(a->π.Q[(s,a)], π.P.A)
end 

function simulate!(π::MonteCarloTreeSearch, s, d=π.d)
    if d <= 0 
        return rollout(π.P, s, π.d_r)
    end 
    P, N, Q, c = π.P, π.N, π.Q, π.c
    A, TR, gamma = P.A, P.TR, P.gamma
    if !haskey(N,(s,first(A)))  # if (s,a) has never been visited
        for a in A
            N[(s,a)] = 0 
            Q[(s,a)] = 0
        end 
        return rollout(π.P, s, π.d_r)   
    end
    
    # if (s,a) has been visited
    a = explore(π,s)
    # println("State before TR in simulate \n", s)
    s_, r = TR(s,a)
    # println("State after TR in simulate \n", s)
    q = r + gamma* simulate!(π, s_, d-1) 
    N[(s,a)] += 1  # +1 for visit count 
    Q[(s,a)] += (q-Q[(s,a)]) / N[(s,a)]  # The more you visited, the update of the Q will (usually) converge 
    return q 
end 

bonus(Nsa, Ns) = Nsa==0 ? Inf : sqrt(log(Ns)/Nsa)

function explore(π::MonteCarloTreeSearch, s)
    A, N, Q, c = π.P.A, π.N, π.Q, π.c
    Ns = sum(N[(s,a)] for a in A)
    # objective = Q+bonus term 
    # if there is no past visit, then that exploration is always prioritized 
    return argmax(a -> Q[(s,a)] + c*bonus(N[(s,a)], Ns), A)  
end 

function feasible_actions(s, A)
    # s: Current state
    # A: Total action space
    new_action_space = [1] # initialize with do nothing
    for action in A
        if action > 1
            rv = koe2cart(copy(s.koe), mu_E)
            target = s.target_list[action-1]
            target_pos = ECEF_to_ECI([target[1], target[2], target[3], 0, 0, 0], s.dt)
            R_eci2rtn = ECI_to_RTN_matrix(rv)
            look_vec_rtn = R_eci2rtn * (target_pos[1:3] .- rv[1:3]) 
            
            horizon_dist = sqrt(norm(rv[1:3])^2 - Re^2)
            far_side = norm(look_vec_rtn) > horizon_dist
            if far_side
            else
                push!(new_action_space, action)
            end
        end
    end
    return new_action_space
end


feasible_actions (generic function with 1 method)

## Run the problem with Heuristic actions

In [13]:
# setup parameters
num_runs = 10
reward_totals = zeros(num_runs)
num_observed = zeros(num_runs)
target_list, n_targets = create_target_list_3d(dataset)

state_list = []
action_list = []

# Initializing state
koe0 = [
    7057.0,  # a [km]
    0.000879,  # e
    deg2rad(98.12),  # i
    deg2rad(255.09),  # Ω [rad]
    deg2rad(225.37),  # ω [rad]
    deg2rad(75.0),  # M [rad]
]
att = [0.,0.]
dt = 0

observed_list = zeros(n_targets)
state = State3d(koe0, att, dt, target_list, observed_list)

for j in 1:num_runs
    A = 1:n_targets+1 

    time_step = 30 # seconds
    num_minutes = 30
    slew_limit = 15

    # Create an intial state
    # x0 = 0 # checl: are the test points centered around x=0?
    # y0 = 0
    # dydt = 1
    # alt = 50
    # for i in 1:num_runs
    koe0 = [
        7057.0,  # a [km]
        0.000879,  # e
        deg2rad(98.12),  # i
        deg2rad(255.09),  # Ω [rad]
        deg2rad(225.37),  # ω [rad]
        deg2rad(75.0),  # M [rad]
    ]
    att = [0.,0.]
    dt = 0
    
    observed_list = zeros(n_targets)
    state = State3d(koe0, att, dt, target_list, observed_list)
    state_list = []
    action_list = []
    for t in 0:time_step:2000
        state_copy = copy(state)
        push!(state_list, state_copy)
        action = heuristic_action(state, slew_limit)
        push!(action_list, action)
        state, reward = TR_orbit(state, action, time_step)
        reward_totals[j] += reward

        # Calculate progress percentage
        progress = t / 2000 * 100

        # Create a string representation of the progress bar
        bar_length = 20
        bar_progress = round(Int, progress / (100 / bar_length))
        bar_string = "[" * repeat("=" , bar_progress) * repeat(" " , bar_length - bar_progress) * "]"

        # Print the progress bar
        print("\rProgress: $bar_string $(round(progress, digits=2))%")

        # Sleep for a short duration to see the progress bar
        sleep(0.1)
    end
    num_observed[j] = sum(state.observed_list)
    println(j)
end

println("Final reward: ", mean(reward_totals))

Progress: [====================] 99.0%1
Progress: [====================] 99.0%2
Progress: [====================] 99.0%3
Progress: [====================] 99.0%4
Progress: [====================] 99.0%5
Progress: [====================] 99.0%6
Progress: [====================] 99.0%7
Progress: [====================] 99.0%8
Progress: [====================] 99.0%9
Progress: [====================] 99.0%10
Final reward: 9.271619889644962


In [15]:
println("Average reward ", mean(reward_totals))
println("Median number of sites observed ", median(num_observed))

Average reward 9.271619889644962
Median number of sites observed 17.0


In [16]:
reward_totals

10-element Vector{Float64}:
  8.557747023353961
  9.34357951055156
  8.634630432065844
 12.468316999132448
 10.12173525988986
  9.424690148459822
  9.436974931085498
  8.125560009185737
  8.498794072642392
  8.104170510082481

In [17]:
df = CSV.read(dataset, DataFrame)
target_lambda = df.lambda
target_phi = df.phi
target_mean_reward = df.r_mean
observed_target_lambda = []
observed_target_phi = []
observed_target_reward = []
for i in 1:length(state.observed_list)
    if state.observed_list[i] == 1
        push!(observed_target_lambda, target_lambda[i])
        push!(observed_target_phi, target_phi[i])
        push!(observed_target_reward, target_mean_reward[i])
    end
end
observed_target_lambda = Float64.(observed_target_lambda)
observed_target_phi = Float64.(observed_target_phi)
observed_target_reward = Float64.(observed_target_reward)

angs_list = []
for i in 1:length(action_list)
    # println(i)
    action = action_list[i]
    # println(action)
    state = state_list[i]
    # println(state)
    if action > 1
        target = state.target_list[action-1]
        push!(angs_list, get_slew_angle(state.koe, target, state.dt))
    else 
        if i > 1
            push!(angs_list, angs_list[i-1])
        else
            push!(angs_list, (0,0))
        end
    end
end
angs_list = collect(angs_list)
cross_ang_des = [t[1] for t in angs_list];
along_ang_des = [t[2] for t in angs_list];

## Run the given problem with MCTS

In [21]:
target_list, n_targets = create_target_list_3d(dataset)
num_runs = 1
mcts_reward_totals = zeros(num_runs)
mcts_num_observed = zeros(num_runs)
target_list, n_targets = create_target_list_3d(dataset)
action_space =  1:n_targets+1 

# MCTS parameters
gamma = 0.95 # discount factor for the MDP problem
m = 500 # simulation count
c = 4000 # exploration constant
d = 30 # simulation depth
d_r = 1 # rollout depth
P = MDP(gamma, action_space, TR_orbit)
time_step = 30

state_list_mcts = []
action_list_mcts = []
# Create an intial state
for j in 1:num_runs
    koe0 = [
        7057.0,  # a [km]
        0.000879,  # e
        deg2rad(98.12),  # i
        deg2rad(255.09),  # Ω [rad]
        deg2rad(225.37),  # ω [rad]
        deg2rad(75.0),  # M [rad]
    ]
    att = [0.,0.]
    dt = 0
    observed_list = zeros(n_targets)
    state = State3d(koe0, att, dt, target_list, observed_list)

    Q = Dict{Tuple{State3d, Int}, Float64}()
    N = Dict{Tuple{State3d, Int}, Int}()
    U = Dict{State, Float64}()

    # Q, N, and U are going to be difficult to define
    mcts_run = MonteCarloTreeSearch(P, N, Q, d, d_r, m, c, U)

    state_list_mcts = []
    action_list_mcts = []
    for t = 0:time_step:2000
        # Push to the full list
        state_copy = copy(state)
        push!(state_list_mcts, state_copy)
        # println("State copy")
        # println(state_copy.observed_list)

        # Run MCTS
        state_for_mcts = copy(state)
        mcts_action = mcts_run(state_for_mcts)

        # Use the action
        push!(action_list_mcts, mcts_action)
        state, reward = TR_orbit(state, mcts_action, time_step)
        mcts_reward_totals[j] += reward

        # Calculate progress percentage
        progress = t / 2000 * 100

        # Create a string representation of the progress bar
        bar_length = 20
        bar_progress = round(Int, progress / (100 / bar_length))
        bar_string = "[" * repeat("=" , bar_progress) * repeat(" " , bar_length - bar_progress) * "]"

        # Print the progress bar
        print("\rProgress: $bar_string $(round(progress, digits=2))%")

        # Sleep for a short duration to see the progress bar
        sleep(0.1)
    end
    mcts_num_observed[j] = sum(state.observed_list)
    println(j)
end
println("Final reward: ", mean(mcts_reward_totals))

Progress: [=============       ] 64.5%

: 

: 

In [ ]:
println("Average reward ", mean(mcts_reward_totals))
println("Median number of sites observed ", median(mcts_num_observed))

In [ ]:
mcts_observed_target_lambda = []
mcts_observed_target_phi = []
mcts_observed_target_reward = []
for i in 1:length(state.observed_list)
    if state.observed_list[i] == 1
        push!(mcts_observed_target_lambda, target_lambda[i])
        push!(mcts_observed_target_phi, target_phi[i])
        push!(mcts_observed_target_reward, target_mean_reward[i])
    end
end
mcts_observed_target_lambda = Float64.(mcts_observed_target_lambda)
mcts_observed_target_phi = Float64.(mcts_observed_target_phi)
mcts_observed_target_reward = Float64.(mcts_observed_target_reward)


angs_list_mcts = []
for i in 1:length(action_list_mcts)
    # println(i)
    action = action_list_mcts[i]
    # println(action)
    state = state_list_mcts[i]
    # println(state)
    if action > 1
        target = state.target_list[action-1]
        push!(angs_list_mcts, get_slew_angle(state.koe, target, state.dt))
    else 
        if i > 1
            push!(angs_list_mcts, angs_list_mcts[i-1])
        else
            push!(angs_list_mcts, (0,0))
        end
    end
end
angs_list_mcts = collect(angs_list_mcts)
cross_ang_des_mcts = [t[1] for t in angs_list_mcts]
along_ang_des_mcts = [t[2] for t in angs_list_mcts];

In [ ]:
mcts_num_observed

## Plotting results

In [ ]:
# x_list = zeros(length(state_list))
# y_list = zeros(length(state_list))
x_angle_list = []
y_angle_list = []
index = 1:length(state_list)
for i in 1:length(state_list)
    # x_list[i] = state_list[i].x
    # y_list[i] = state_list[i].y
    push!(x_angle_list, state_list[i].attitude[1])
    push!(y_angle_list, state_list[i].attitude[2])
end

# x_list_mcts = zeros(length(state_list_mcts))
# y_list_mcts = zeros(length(state_list_mcts))
x_angle_list_mcts = []
y_angle_list_mcts = []
index_mcts = 1:length(state_list_mcts)
for i in 1:length(state_list_mcts)
#     x_list_mcts[i] = state_list_mcts[i].x
#     y_list_mcts[i] = state_list_mcts[i].y
    push!(x_angle_list_mcts, state_list_mcts[i].attitude[1])
    push!(y_angle_list_mcts, state_list_mcts[i].attitude[2])
end


In [ ]:
Plots.plot(index, action_list, label="Heuristic")
Plots.plot!(index, action_list_mcts, title="Actions with MCTS", label="MCTS")


In [ ]:
# plot(index, [y_angle_list, y_angle_list_mcts], label=["Random" "With MCTS"], title="Y angle")
random_along = Plots.plot(index, [y_angle_list, along_ang_des], label=["Actual Angle" "Desired Angle"], title="Along track angle for Baseline")
mcts_along = Plots.plot(index, [y_angle_list_mcts, along_ang_des_mcts], label=["Actual Angle" "Desired Angle"], title="Along track angle for MCTS")
Plots.plot(random_along, mcts_along,layout=(1, 2), size=(1000, 400), ylabel="Degrees", xlabel="Time (s)")
# save()
# plot(index, along_ang_des, label="Desired Random", title="Along Track angle")

In [ ]:
# plot(index, [y_angle_list, y_angle_list_mcts], label=["Random" "With MCTS"], title="Y angle")
random_along = Plots.plot(index, [x_angle_list, cross_ang_des], label=["State Heuristic" "Desired Heuristic"])
mcts_along = Plots.plot(index, [x_angle_list_mcts, cross_ang_des_mcts], label=["State MCTS" "Desired MCTS"])
Plots.plot(random_along, mcts_along,layout=(1, 2), title="Cross track angle", size=(800, 400))
# plot(index, along_ang_des, label="Desired Random", title="Along Track angle")

### Globe plots

In [ ]:
koe = [state.koe for state in state_list]
dts = [state.dt for state in state_list]
state_ECEF = zeros(6, length(dts))
state_ECI = zeros(6, length(dts))
state_lambda = zeros(length(dts))
state_phi = zeros(length(dts))
for (i, koe) in enumerate(koe)
    rv = koe2cart(koe, mu_E)
    rv_ecef = ECI_to_ECEF(rv, dts[i])
    state_lambda[i] = atand(rv_ecef[2], rv_ecef[1])
    state_phi[i] = atand(rv_ecef[3], sqrt(rv_ecef[1]^2 + rv_ecef[2]^2))
end

layout = @layout [a b]
f = Figure(size = (1000, 800), layout=layout)
lon,lat,data = GeoDatasets.landseamask(;resolution='c',grid=5)
ax1 = Axis(f[1, 1], aspect=DataAspect(), title = "Heuristic")
sca1 =GLMakie.scatter!(ax1, target_lambda, target_phi, overdraw = true, color = :grey, markersize = 25*target_mean_reward)
sca2 =GLMakie.scatter!(ax1, observed_target_lambda , observed_target_phi, overdraw = true, color = :blue, markersize = 25*observed_target_reward)
GLMakie.contour!(ax1, lon, lat, data, levels=[0.5], color=:black, linewidth=0.5)
path = lines!(ax1, state_lambda, state_phi, overdraw = true, color = :blue, linewidth = 2,)
GLMakie.ylims!(ax1, low = -66, high = 66)
GLMakie.xlims!(ax1, low = -140, high = -85)
# save(f, "globe.png")

mcts_koe = [state.koe for state in state_list_mcts]
mcts_dts = [state.dt for state in state_list_mcts]
state_ECEF = zeros(6, length(mcts_dts))
state_ECI = zeros(6, length(mcts_dts))
mcts_state_lambda = zeros(length(mcts_dts))
mcts_state_phi = zeros(length(mcts_dts))
for (i, koe) in enumerate(mcts_koe)
    rv = koe2cart(koe, mu_E)
    rv_ecef = ECI_to_ECEF(rv, mcts_dts[i])
    mcts_state_lambda[i] = atand(rv_ecef[2], rv_ecef[1])
    mcts_state_phi[i] = atand(rv_ecef[3], sqrt(rv_ecef[1]^2 + rv_ecef[2]^2))
end


k = 10.580 
Δλ = 10
p_br = [-110 + k + Δλ, -60]
p_bl = [-110 + k - Δλ, -60]
p_tr = [-110 - k + Δλ,  60]
p_tl = [-110 - k - Δλ,  60]

l1 = lines!(ax1, [p_br[1], p_bl[1]], [p_br[2], p_bl[2]], overdraw=true, color=:orange, linestyle=:solid, linewidth=2,)
l2 = lines!(ax1, [p_tr[1], p_tl[1]], [p_tr[2], p_tl[2]], overdraw=true, color=:orange, linestyle=:solid, linewidth=2,)
l3 = lines!(ax1, [p_br[1], p_tr[1]], [p_br[2], p_tr[2]], overdraw=true, color=:orange, linestyle=:solid, linewidth=2,)
l4 = lines!(ax1, [p_bl[1], p_tl[1]], [p_bl[2], p_tl[2]], overdraw=true, color=:orange, linestyle=:solid, linewidth=2,)


lon,lat,data = GeoDatasets.landseamask(;resolution='c',grid=5)
ax2 = Axis(f[1, 2], aspect=DataAspect(), title = "MCTS with Random Rollouts")
GLMakie.scatter!(ax2, target_lambda, target_phi, overdraw = true, color = :grey, markersize = 25*target_mean_reward)
GLMakie.scatter!(ax2, mcts_observed_target_lambda , mcts_observed_target_phi, overdraw = true, color = :blue, markersize = 25*mcts_observed_target_reward)
GLMakie.contour!(ax2, lon, lat, data, levels=[0.5], color=:black, linewidth=0.5)
lines!(ax2, mcts_state_lambda, mcts_state_phi, overdraw = true, color = :blue, linewidth = 2,)
GLMakie.ylims!(ax2, low = -66, high = 66)
GLMakie.xlims!(ax2, low = -140, high = -85)
# legend_position = Point(5.5, 0.5)  

l1 = lines!(ax2, [p_br[1], p_bl[1]], [p_br[2], p_bl[2]], overdraw=true, color=:orange, linestyle=:solid, linewidth=2,)
l2 = lines!(ax2, [p_tr[1], p_tl[1]], [p_tr[2], p_tl[2]], overdraw=true, color=:orange, linestyle=:solid, linewidth=2,)
l3 = lines!(ax2, [p_br[1], p_tr[1]], [p_br[2], p_tr[2]], overdraw=true, color=:orange, linestyle=:solid, linewidth=2,)
l4 = lines!(ax2, [p_bl[1], p_tl[1]], [p_bl[2], p_tl[2]], overdraw=true, color=:orange, linestyle=:solid, linewidth=2,)

legend = Legend(f[1,3], [sca1, sca2, path, l1], ["Unobserved targets", "Observed targets", "Satellite Path", "Target Bounding Box"], framevisible=true)

# Adjust these values as needed
# legend!(f, ax2, legend, position=legend_position)
display(f)
# save(f, "mcts_globe.png")



In [24]:
dataset_50 = "../src/obs_site_Earth_50.csv"
dataset_100 = "../src/obs_site_Earth_100.csv"
dataset_1000 = "../src/obs_site_Earth2.csv"


df_50 = CSV.read(dataset_50, DataFrame)
df_100 = CSV.read(dataset_100, DataFrame)
df_1000 = CSV.read(dataset_1000, DataFrame)
target_lambda_50 = df_50.lambda
target_phi_50 = df_50.phi
target_mean_reward_50 = df_50.r_mean
target_lambda_100 = df_100.lambda
target_phi_100 = df_100.phi
target_mean_reward_100 = df_100.r_mean
target_lambda_1000 = df_1000.lambda
target_phi_1000 = df_1000.phi
target_mean_reward_1000 = df_1000.r_mean;

In [26]:
fontsize_theme = Theme(fontsize = 25)
set_theme!(fontsize_theme)
layout = @layout [a b c]
f = Figure(size = (1200, 800), layout=layout)
lon,lat,data = GeoDatasets.landseamask(;resolution='c',grid=5)
ax1 = Axis(f[1, 1], aspect=DataAspect(), title = "50 Targets")
ax2 = Axis(f[1, 2], aspect=DataAspect(), title = "100 Targets")
ax3 = Axis(f[1, 3], aspect=DataAspect(), title = "1000 Targets")
GLMakie.scatter!(ax1, target_lambda_50, target_phi_50, overdraw = true, color = :red, markersize = 20*target_mean_reward_50)
GLMakie.contour!(ax1, lon, lat, data, levels=[0.5], color=:black, linewidth=0.5)
GLMakie.ylims!(ax1, low = -66, high = 66)
GLMakie.xlims!(ax1, low = -140, high = -85)
GLMakie.scatter!(ax2, target_lambda_100, target_phi_100, overdraw = true, color = :red, markersize = 20*target_mean_reward_100)
GLMakie.contour!(ax2, lon, lat, data, levels=[0.5], color=:black, linewidth=0.5)
GLMakie.ylims!(ax2, low = -66, high = 66)
GLMakie.xlims!(ax2, low = -140, high = -85)
GLMakie.scatter!(ax3, target_lambda_1000, target_phi_1000, overdraw = true, color = :red, markersize = 20*target_mean_reward_1000)
GLMakie.contour!(ax3, lon, lat, data, levels=[0.5], color=:black, linewidth=0.5)
GLMakie.ylims!(ax3, low = -66, high = 66)
GLMakie.xlims!(ax3, low = -140, high = -85)
k = 10.580 
Δλ = 10
p_br = [-110 + k + Δλ, -60]
p_bl = [-110 + k - Δλ, -60]
p_tr = [-110 - k + Δλ,  60]
p_tl = [-110 - k - Δλ,  60]
l1 = lines!(ax1, [p_br[1], p_bl[1]], [p_br[2], p_bl[2]], overdraw=true, color=:orange, linestyle=:solid, linewidth=2,)
l2 = lines!(ax1, [p_tr[1], p_tl[1]], [p_tr[2], p_tl[2]], overdraw=true, color=:orange, linestyle=:solid, linewidth=2,)
l3 = lines!(ax1, [p_br[1], p_tr[1]], [p_br[2], p_tr[2]], overdraw=true, color=:orange, linestyle=:solid, linewidth=2,)
l4 = lines!(ax1, [p_bl[1], p_tl[1]], [p_bl[2], p_tl[2]], overdraw=true, color=:orange, linestyle=:solid, linewidth=2,)
l5 = lines!(ax2, [p_br[1], p_bl[1]], [p_br[2], p_bl[2]], overdraw=true, color=:orange, linestyle=:solid, linewidth=2,)
l6 = lines!(ax2, [p_tr[1], p_tl[1]], [p_tr[2], p_tl[2]], overdraw=true, color=:orange, linestyle=:solid, linewidth=2,)
l7 = lines!(ax2, [p_br[1], p_tr[1]], [p_br[2], p_tr[2]], overdraw=true, color=:orange, linestyle=:solid, linewidth=2,)
l8 = lines!(ax2, [p_bl[1], p_tl[1]], [p_bl[2], p_tl[2]], overdraw=true, color=:orange, linestyle=:solid, linewidth=2,)
l9 = lines!(ax3, [p_br[1], p_bl[1]], [p_br[2], p_bl[2]], overdraw=true, color=:orange, linestyle=:solid, linewidth=2,)
l10 = lines!(ax3, [p_tr[1], p_tl[1]], [p_tr[2], p_tl[2]], overdraw=true, color=:orange, linestyle=:solid, linewidth=2,)
l11 = lines!(ax3, [p_br[1], p_tr[1]], [p_br[2], p_tr[2]], overdraw=true, color=:orange, linestyle=:solid, linewidth=2,)
l12 = lines!(ax3, [p_bl[1], p_tl[1]], [p_bl[2], p_tl[2]], overdraw=true, color=:orange, linestyle=:solid, linewidth=2,)


display(f)

GLMakie.Screen(...)

### OLD PLots

In [ ]:
# plt = plot(x_list, y_list, label="Position", aspect_ratio=:equal, legend=:bottomleft, dpi=500)

# for i in 1:length(x_list)
#     if action_list[i] != 1
#         plot!([x_list[i], target_x_r[action_list[i] - 1]], [y_list[i], target_y_r[action_list[i] - 1]], label="", c="gray")
#     end
# end

# reward_round = round(reward_total, digits=2)
# text_to_display = "Reward: $reward_round"
# annotate!((-35, 30, text(text_to_display, 10, :left, :center, :black)))


# scatter!(target_x_r,target_y_r, mc="green", label="Unobserved targets", markersize = 10*target_reward_r)
# scatter!(observed_target_x_r,observed_target_y_r, mc="blue", label="Observed targets", markersize = 10*observed_target_reward_r)
# display(plt)

# savefig(plt, "random_targeting.png")


In [ ]:
# plt = plot(x_list_mcts, y_list_mcts, label="Position", aspect_ratio=:equal, legend=:bottomleft, dpi=500)

# for i in 1:length(x_list_mcts)
#     if action_list_mcts[i] != 1
#         plot!([x_list_mcts[i], target_x[action_list_mcts[i] - 1]], [y_list_mcts[i], target_y[action_list_mcts[i] - 1]], label="", c="gray")
#     end
# end

# reward_mcts_round = round(reward_total_mcts, digits=2)
# text_to_display = "Reward: $reward_mcts_round"
# annotate!((-35, 30, text(text_to_display, 10, :left, :center, :black)))

# scatter!(target_x,target_y, mc="green", label="Unobserved targets", markersize = 10*target_reward)
# scatter!(observed_target_x,observed_target_y, mc="blue", label="Observed targets", markersize = 10*observed_target_reward)

# display(plt)
# savefig(plt, "mcts_targeting.png")